In [1]:
import numpy as np
import scipy as sp
from scipy import linalg as LA
import matplotlib.pyplot as plt
import time as tm
import ipywidgets as widgets

In [2]:
#Define the problem domain (1-D) and discretization / constants
xmax = 50 #m
n = 31
x = np.linspace(0,xmax,num=n)
dx = xmax/n-1 #m
dy = 1 #m
dz = 10 #m
dt = 5 #s
K = 3E-7 #m/s
Ss = 2E-5 #[m^-1]
sim_time = 60*60 #s
num_tstep = int(np.ceil(sim_time/dt)) #number of time steps including the first

#Boundary conditions - AFTER reservoir head change
h_L = 16.
h_R = 11.

#Initial condition
h_init = h_L*np.ones((1,n))
h_init[0,-1] = h_R

h_soln_exp = np.zeros((num_tstep,n)) 
h_soln_imp = np.zeros((num_tstep,n))

In [3]:
#Explicit approach: Setup the matrix system and solve

D = K*dt/(Ss*dx**2)
print('D = ', D)

h_soln_exp[0,:] = h_init
tstep = 1
while tstep < num_tstep:
  for c in range(n):
    if c == 0:
      h_soln_exp[tstep,c] = h_L
    elif c == (n-1):
      h_soln_exp[tstep,c] = h_R
    else:
      h_soln_exp[tstep,c] = h_soln_exp[tstep-1,c] + \
        D*(h_soln_exp[tstep-1,c-1] - 2*h_soln_exp[tstep-1,c] + \
           h_soln_exp[tstep-1,c+1])
  tstep += 1



D =  0.19965373961218844


In [4]:
#Implicit approach: Setup the matrix system, check the formulation

D = K*dt/(Ss*dx**2)

h_soln_imp[0,:] = h_init

tstep = 1

while tstep < num_tstep:
  A_step = np.zeros((n,n))
  b_step = np.zeros((n))
  for c in range(n):
    if c == 0:
      A_step[c,c] = 1
      b_step[c] = h_L
    elif c == (n-1):
      A_step[c,c] = 1
      b_step[c] = h_R
    else:
      A_step[c,c-1] = -D
      A_step[c,c] = (1+2*D)
      A_step[c,c+1] = -D
      b_step[c] = h_soln_imp[tstep-1,c]
  h_new = LA.solve(A_step,b_step)
  h_soln_imp[tstep,:] = h_new
  tstep += 1


In [5]:
#Create function to plot the explicit solution and use iPython Widgets to visualize

def plot_exp(tstep):
    fig, axes = plt.subplots(1, 1, figsize=(8, 3))
    axes.plot(x, h_soln_exp[tstep,:])
    axes.set_title(f'Time = {tstep*dt}')

## Generate our user interface.
widgets.interact(plot_exp, tstep=widgets.IntSlider(0,min=0,max=(num_tstep-1)));

interactive(children=(IntSlider(value=0, description='tstep', max=719), Output()), _dom_classes=('widget-inter…

In [6]:
#Create function to plot the implicit solution and use iPython Widgets to visualize

def plot_imp(tstep):
    fig, axes = plt.subplots(1, 1, figsize=(8, 3))
    axes.plot(x, h_soln_imp[tstep,:])
    axes.set_title(f'Time = {tstep*dt}')
    print(tstep*dt)

## Generate our user interface.
widgets.interact(plot_imp, tstep=widgets.IntSlider(0,min=0,max=(num_tstep-1)));

interactive(children=(IntSlider(value=0, description='tstep', max=719), Output()), _dom_classes=('widget-inter…